# 1. FeatureExtraction_test

Reference:
- https://www.kaggle.com/asraful70/talkingdata-new-features-in-lightgbm-lb-0-9784
- https://www.kaggle.com/danieleewww/talkingdata-added-new-features-in-lightg-50cf9b/code
- https://www.kaggle.com/anttip/talkingdata-wordbatch-fm-ftrl-lb-0-9769
- https://www.kaggle.com/pranav84/talkingdata-eda-to-model-evaluation-lb-0-9683
- https://www.kaggle.com/aharless/kaggle-runnable-version-of-baris-kanber-s-lightgbm
- https://www.kaggle.com/pranav84/lgb-entire-dataset-in-2-hrs-lb-0-9718
- https://www.kaggle.com/panjianning/talkingdata-simple-lightgbm-0-9772

## Run name

In [1]:
import time

project_name = 'TalkingdataAFD2018'
step_name = 'FeatureExtraction_test'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = '%s_%s_%s' % (project_name, step_name, time_str)
print('run_name: %s' % run_name)
t0 = time.time()

run_name: TalkingdataAFD2018_FeatureExtraction_test_20180501_143516


## Important params

In [2]:
# date = 0
# print('date: ', date)

is_debug = False
print('is_debug: %s' % is_debug)


if is_debug:
    test_n_rows = 1 * 10000
else:
    test_n_rows = None
#     test_n_rows = 18790469

is_debug: False


In [3]:
day_rows = {
    0: {
        'n_skiprows': 1,
        'n_rows': 1 * 10000
    },
    1: {
        'n_skiprows': 1 * 10000,
        'n_rows': 2 * 10000
    },
    6: {
        'n_skiprows': 1,
        'n_rows': 9308568
    },
    7: {
        'n_skiprows': 1 + 9308568,
        'n_rows': 59633310
    },
    8: {
        'n_skiprows': 1 + 9308568 + 59633310,
        'n_rows': 62945075
    },
    9: {
        'n_skiprows': 1 + 9308568 + 59633310 + 62945075,
        'n_rows': 53016937
    }
}
# n_skiprows = day_rows[date]['n_skiprows']
# n_rows = day_rows[date]['n_rows']

## Import PKGs

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import time
import random
import zipfile
import h5py
import pickle
import math
from PIL import Image
import shutil

from tqdm import tqdm
import multiprocessing
from multiprocessing import cpu_count

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

random_num = np.random.randint(10000)
print('random_num: %s' % random_num)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


random_num: 7148


## Project folders

In [5]:
cwd = os.getcwd()

input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t%s' % input_folder)
print('output_folder: \t\t\t%s' % output_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('feature_folder: \t\t%s' % feature_folder)
print('log_folder: \t\t\t%s' % log_folder)

train_csv_file = os.path.join(input_folder, 'train.csv')
train_sample_csv_file = os.path.join(input_folder, 'train_sample.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_csv_file = os.path.join(input_folder, 'sample_submission.csv')

print('\ntrain_csv_file: \t\t%s' % train_csv_file)
print('train_sample_csv_file: \t\t%s' % train_sample_csv_file)
print('test_csv_file: \t\t\t%s' % test_csv_file)
print('sample_submission_csv_file: \t%s' % sample_submission_csv_file)

input_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/input
output_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/output
model_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/model
feature_folder: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/feature
log_folder: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/log

train_csv_file: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/input/train.csv
train_sample_csv_file: 		/data1/kaggle/talkingdata-adtracking-fraud-detection/input/train_sample.csv
test_csv_file: 			/data1/kaggle/talkingdata-adtracking-fraud-detection/input/test.csv
sample_submission_csv_file: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/input/sample_submission.csv


## Load data

In [6]:
train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']
dtypes = {
    'ip'            : 'uint32',
    'app'           : 'uint16',
    'device'        : 'uint16',
    'os'            : 'uint16',
    'channel'       : 'uint16',
    'is_attributed' : 'uint8',
    'click_id'      : 'uint32'
}

In [7]:
sample_submission_csv = pd.read_csv(sample_submission_csv_file)
print('sample_submission_csv.shape: \t', sample_submission_csv.shape)
display(sample_submission_csv.head(2))

print('train_csv: %.2f Mb' % (sys.getsizeof(sample_submission_csv)/1024./1024.))

sample_submission_csv.shape: 	 (18790469, 2)


,click_id,is_attributed
0,0,0
1,1,0


train_csv: 286.72 Mb


In [8]:
def do_click_time(df):
    df['day'] = df['click_time'].dt.day.astype('uint8')
    df['hour'] = df['click_time'].dt.hour.astype('uint8')
    df['minute'] = df['click_time'].dt.minute.astype('uint8')
    df['second'] = df['click_time'].dt.second.astype('uint8')
    return df

In [9]:
def do_prev_click(df, group_cols, agg_type='float32'):
    agg_suffix = 'prevClick'
    new_feature = new_feature = '{}_{}'.format('_'.join(group_cols), agg_suffix)
    all_features = group_cols + ['click_time']
    df[new_feature] = (df.click_time - df[all_features].groupby(group_cols).click_time.shift(+1) ).dt.seconds.astype(agg_type)
    return df
    
def do_next_click(df, group_cols, agg_type='float32'):
    agg_suffix = 'nextClick'
    new_feature = new_feature = '{}_{}'.format('_'.join(group_cols), agg_suffix)
    all_features = group_cols + ['click_time']
    df[new_feature] = (df[all_features].groupby(group_cols).click_time.shift(-1) - df.click_time).dt.seconds.astype(agg_type)
    return df

In [10]:
## Below a function is written to extract count feature by aggregating different cols
def do_count( df, group_cols, agg_type='uint32', show_max=False, show_agg=True ):
    agg_name='{}_count'.format('_'.join(group_cols))
    if show_agg:
        print( "Aggregating by ", group_cols ,  '... and saved in', agg_name )
    gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
#     predictors.append(agg_name)
#     print('predictors',predictors)
    gc.collect()
    return( df )
    
##  Below a function is written to extract unique count feature from different cols
def do_countuniq( df, group_cols, counted, agg_type='uint32', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_countuniq'.format(('_'.join(group_cols)),(counted))  
    if show_agg:
        print( "Counting unqiue ", counted, " by ", group_cols ,  '... and saved in', agg_name )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
#     predictors.append(agg_name)
#     print('predictors',predictors)
    gc.collect()
    return( df )
### Below a function is written to extract cumulative count feature  from different cols    
def do_cumcount( df, group_cols, counted,agg_type='uint32', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_cumcount'.format(('_'.join(group_cols)),(counted)) 
    if show_agg:
        print( "Cumulative count by ", group_cols , '... and saved in', agg_name  )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].cumcount()
    df[agg_name]=gp.values
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
#     predictors.append(agg_name)
#     print('predictors',predictors)
    gc.collect()
    return( df )
### Below a function is written to extract mean feature  from different cols
def do_mean( df, group_cols, counted, agg_type='float32', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_mean'.format(('_'.join(group_cols)),(counted))  
    if show_agg:
        print( "Calculating mean of ", counted, " by ", group_cols , '... and saved in', agg_name )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
#     predictors.append(agg_name)
#     print('predictors',predictors)
    gc.collect()
    return( df )

def do_var( df, group_cols, counted, agg_type='float32', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_var'.format(('_'.join(group_cols)),(counted)) 
    if show_agg:
        print( "Calculating variance of ", counted, " by ", group_cols , '... and saved in', agg_name )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].var().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
#     predictors.append(agg_name)
#     print('predictors',predictors)
    gc.collect()
    return( df )

In [11]:
def save_feature(x_data, y_data, file_name):
    print(y_data[:5])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: \t%s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('x_data', data=x_data)
        h.create_dataset('y_data', data=y_data)
    print('File saved: \t%s' % file_name)

def load_feature(file_name):
    with h5py.File(file_name, 'r') as h:
        x_data = np.array(h['x_data'])
        y_data = np.array(h['y_data'])
    print('File loaded: \t%s' % file_name)
    print(y_data[:5])
    
    return x_data, y_data


def save_test_feature(x_test, click_ids, file_name):
    print(click_ids[:5])
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: \t%s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('x_test', data=x_test)
        h.create_dataset('click_ids', data=click_ids)
    print('File saved: \t%s' % file_name)

def load_test_feature(file_name):
    with h5py.File(file_name, 'r') as h:
        x_test = np.array(h['x_test'])
        click_ids = np.array(h['click_ids'])
    print('File loaded: \t%s' % file_name)
    print(click_ids[:5])
    
    return x_test, click_ids

In [12]:
def save_feature_map(feature_map, file_name):
    print(feature_map[:5])
    feature_map_encode = []
    for item in feature_map:
        feature_name_encode = item[1].encode('UTF-8')
        feature_map_encode.append((item[0], feature_name_encode))
    if os.path.exists(file_name):
        os.remove(file_name)
        print('File removed: \t%s' % file_name)
    with h5py.File(file_name) as h:
        h.create_dataset('feature_map', data=feature_map_encode)
    print('File saved: \t%s' % file_name)

def load_feature_map(file_name):
    with h5py.File(file_name, 'r') as h:
        feature_map_encode = np.array(h['feature_map'])
    print('File loaded: \t%s' % file_name)
    feature_map = []
    for item in feature_map_encode:
        feature_name = item[1].decode('UTF-8')
        feature_map.append((int(item[0]), feature_name))
    print(feature_map[:5])
    
    return feature_map

In [13]:
def do_feature(train_csv):
    train_csv = do_click_time(train_csv)
    
    for cols in do_prev_click_cols:
        print('>> ', cols)
        train_csv = do_prev_click( train_csv, cols ); gc.collect()
    
    for cols in do_next_click_cols:
        print('>> ', cols)
        train_csv = do_next_click( train_csv, cols ); gc.collect()
    
    for cols in do_count_cols:
        print('>> ', cols)
        train_csv = do_count( train_csv, cols ); gc.collect()
        
    for cols in do_countuniq_cols:
        print('>> ', cols[:-1], cols[-1])
        train_csv = do_countuniq( train_csv, cols[:-1], cols[-1] ); gc.collect()
        
    for cols in do_cumcount_cols:
        print('>> ', cols[:-1], cols[-1])
        train_csv = do_cumcount( train_csv, cols[:-1], cols[-1] ); gc.collect()
        
    for cols in do_mean_cols:
        print('>> ', cols[:-1], cols[-1])
        train_csv = do_mean( train_csv, cols[:-1], cols[-1] ); gc.collect()
        
    for cols in do_var_cols:
        print('>> ', cols[:-1], cols[-1])
        train_csv = do_var( train_csv, cols[:-1], cols[-1] ); gc.collect()
    
    train_csv.drop(['click_time'], axis=1, inplace=True)
    print(train_csv.shape)

    display(train_csv.head())

    print(train_csv.columns)
    print('data_size: %.2f Mb' % (sys.getsizeof(train_csv)/1024./1024.))
    return train_csv

In [14]:
template = [
    # 5 choice 2
    ['ip', 'app'],
    ['ip', 'device'],
    ['ip', 'os'],
    ['ip', 'channel'],
    ['app', 'device'],
    ['app', 'os'],
    ['app', 'channel'],
    ['device', 'os'],
    ['device', 'channel'],
    ['os', 'channel'],
    # 5 choice 3
    ['device', 'os', 'channel'],
    ['app', 'os', 'channel'],
    ['app', 'device', 'channel'],
    ['app', 'device', 'os'],
    ['ip', 'os', 'channel'],
    ['ip', 'device', 'channel'],
    ['ip', 'device', 'os'],
    ['ip', 'app', 'channel'],
    ['ip', 'app', 'os'],
    ['ip', 'app', 'device'],
]

template_hour = [
    # 5 choice 2
    ['ip', 'app', 'hour'],
    ['ip', 'device', 'hour'],
    ['ip', 'os', 'hour'],
    ['ip', 'channel', 'hour'],
    ['app', 'device', 'hour'],
    ['app', 'os', 'hour'],
    ['app', 'channel', 'hour'],
    ['device', 'os', 'hour'],
    ['device', 'channel', 'hour'],
    ['os', 'channel', 'hour'],
    # 5 choice 3
    ['device', 'os', 'channel', 'hour'],
    ['app', 'os', 'channel', 'hour'],
    ['app', 'device', 'channel', 'hour'],
    ['app', 'device', 'os', 'hour'],
    ['ip', 'os', 'channel', 'hour'],
    ['ip', 'device', 'channel', 'hour'],
    ['ip', 'device', 'os', 'hour'],
    ['ip', 'app', 'channel', 'hour'],
    ['ip', 'app', 'os', 'hour'],
    ['ip', 'app', 'device', 'hour'],
]

In [15]:
do_prev_click_cols = [
    ['ip', 'device'],
    ['ip', 'app', 'device'],
    ['ip', 'app'],
    ['ip', 'app', 'device', 'os'],
    ['ip', 'app', 'device', 'os', 'channel'],
    ['ip', 'app', 'os', 'channel'],
    ['ip', 'device', 'os', 'channel'],
    
    
    
    ['ip', 'os'],
    ['ip', 'device', 'channel'],
    ['ip', 'channel'], # ref
]

do_next_click_cols = [
    ['ip', 'device'],
    ['ip', 'app', 'device'],
    ['ip', 'app'],
    ['ip', 'app', 'device', 'os'], # ref
    ['ip', 'os'],
    ['ip', 'device', 'os'], # ref
    ['ip', 'device', 'os', 'channel'],
    ['ip', 'os', 'channel'],
    ['ip', 'app', 'os', 'channel'],
    
    ['ip', 'app', 'os'],
    ['ip', 'device', 'channel'],
    
    ['ip', 'app', 'device', 'os', 'channel'], # ref
    ['device', 'channel'], # ref
    ['app', 'device', 'channel'], # ref
    ['device', 'hour'], # ref
    
#     ['ip', 'device'],
#     ['ip', 'app', 'device', 'channel'],
#     ['ip', 'os'],
#     ['ip', 'app', 'channel'],
#     ['ip' ,'channel'],
]

do_count_cols = [
    ['ip', 'device'],
    ['app', 'channel'],
    ['device', 'os', 'channel', 'hour'],
    ['ip', 'device', 'hour'],
    ['app', 'device', 'os'],
    ['app', 'os', 'channel', 'hour'],
    ['app', 'os'],
    
    
    ['app', 'hour'],
    ['ip', 'day', 'hour'], # ref
    ['ip', 'app'], # ref
    ['ip', 'app', 'os'], # ref
]
do_countuniq_cols = [
    ['ip', 'app'], # ref
    ['ip', 'device', 'channel'],
    ['ip', 'device', 'os'], # ref
    
    
    ['ip', 'channel'], # ref
    ['ip', 'device', 'os', 'hour'],
    
    ['ip', 'day', 'hour'], # ref
    ['ip', 'app', 'os'], # ref
    ['ip', 'device'],
    ['app', 'channel'],
]
do_cumcount_cols = [
    ['app', 'os', 'hour'],
    ['app', 'device', 'channel'],
    ['app', 'device'],
    ['app', 'device', 'os'],
    ['device', 'os'],
    
    
    ['app', 'channel', 'hour'],
    ['os', 'channel'],
    ['device', 'channel', 'hour'],
    ['device', 'os', 'channel'],
    ['os', 'channel', 'hour'],
    ['device', 'os', 'hour'],
    ['app', 'device', 'channel', 'hour'],
    ['app', 'os', 'channel'],
    
    ['ip', 'os'], # ref
    ['ip', 'device', 'os'], # ref
]
do_mean_cols = [
    ['ip', 'app'],
    ['ip', 'app', 'channel'],
    ['ip', 'os', 'channel'],
    ['ip', 'device', 'os'],
    ['ip', 'os'],
    ['ip', 'device', 'hour'],
    ['ip', 'channel'],
    ['ip', 'app', 'os'],
    ['ip', 'device', 'channel'],
    ['os', 'channel', 'hour'],
    ['app', 'os', 'channel'],
    ['device', 'channel', 'hour'],
    ['ip', 'app', 'channel', 'hour'],
    ['ip', 'app', 'hour'],
    ['ip', 'os', 'hour'],
    ['ip', 'device', 'os', 'hour'],
    ['ip', 'os', 'channel', 'hour'],
    ['app', 'channel', 'hour'],
    ['app', 'device', 'os', 'hour'],
    

    
#     ['ip', 'os', 'channel'],
#     ['ip', 'app', 'os'],
#     ['ip', 'device', 'channel']
]
do_var_cols = [
    ['ip', 'os', 'channel'],
    ['ip', 'app'],
    ['ip', 'app', 'channel'],
    ['ip', 'device', 'hour'],
    ['ip', 'device', 'channel'],
    ['ip', 'app', 'os'],
    ['ip', 'device', 'os'],
    ['ip', 'channel'],
    ['ip', 'os'],
    ['app', 'os', 'channel', 'hour'],
    ['ip', 'device', 'os', 'hour'],
    ['device', 'os', 'channel', 'hour'],
    ['os', 'channel'],
    ['app', 'channel', 'hour'],
    ['ip', 'device', 'channel', 'hour'],
    ['ip', 'app', 'device'],
    ['app', 'os', 'hour'],
    
    ['ip', 'app', 'hour'],
    ['ip', 'app', 'device', 'hour'],
#     ['ip', 'os', 'hour'],
    ['app', 'os', 'channel'],
#     ['ip', 'channel']
]

for cols in do_count_cols:
    print(cols[:-1], cols[-1])

feature_files = []

['ip'] device
['app'] channel
['device', 'os', 'channel'] hour
['ip', 'device'] hour
['app', 'device'] os
['app', 'os', 'channel'] hour
['app'] os
['app'] hour
['ip', 'day'] hour
['ip'] app
['ip', 'app'] os


In [16]:
%%time

test_csv = pd.read_csv(
    test_csv_file, 
    nrows=test_n_rows, 
    usecols=test_columns,
    dtype=dtypes,
    parse_dates=['click_time']
)

CPU times: user 17.9 s, sys: 708 ms, total: 18.6 s
Wall time: 47.6 s


In [17]:
%%time

print('test_csv.shape: \t\t', test_csv.shape)
display(test_csv.head(2))
print('test_csv:  %.2f Mb' % (sys.getsizeof(test_csv)/1024./1024.))
# print('*' * 80)

click_ids = test_csv['click_id']
test_csv.drop(['click_id'], axis=1, inplace=True)
display(click_ids.head())

display(test_csv.head())
# print('*' * 80)

test_csv = do_feature(test_csv)
    
y_proba_file = os.path.join(feature_folder, 'feature_%s_test.p' % run_name)
feature_files.append(y_proba_file)
save_test_feature(
    test_csv, 
    click_ids, 
    y_proba_file
)
x_test, click_ids = load_test_feature(y_proba_file)

print(x_test.shape)
print(len(click_ids))

feature_map = []
print('[')
for i, col in enumerate(test_csv.columns):
    feature_map.append((i, col))
    print('  (%s,\t"%s")' % (i, col))
print(']')
feature_map_file_name = y_proba_file = os.path.join(feature_folder, 'feature_map_%s.p' % run_name)
save_feature_map(feature_map, feature_map_file_name)
feature_map1 = load_feature_map(feature_map_file_name)
print(len(feature_map1))
print(feature_map1[:5])

# del test_csv
# del x_test
# del click_ids
# gc.collect()

test_csv.shape: 		 (18790469, 7)


,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00


test_csv:  430.08 Mb


0    0
1    1
2    2
3    3
4    4
Name: click_id, dtype: uint32

,ip,app,device,os,channel,click_time
0,5744,9,1,3,107,2017-11-10 04:00:00
1,119901,9,1,3,466,2017-11-10 04:00:00
2,72287,21,1,19,128,2017-11-10 04:00:00
3,78477,15,1,13,111,2017-11-10 04:00:00
4,123080,12,1,13,328,2017-11-10 04:00:00


>>  ['ip', 'device']
>>  ['ip', 'app', 'device']
>>  ['ip', 'app']
>>  ['ip', 'app', 'device', 'os']
>>  ['ip', 'app', 'device', 'os', 'channel']
>>  ['ip', 'app', 'os', 'channel']
>>  ['ip', 'device', 'os', 'channel']
>>  ['ip', 'os']
>>  ['ip', 'device', 'channel']
>>  ['ip', 'channel']
>>  ['ip', 'device']
>>  ['ip', 'app', 'device']
>>  ['ip', 'app']
>>  ['ip', 'app', 'device', 'os']
>>  ['ip', 'os']
>>  ['ip', 'device', 'os']
>>  ['ip', 'device', 'os', 'channel']
>>  ['ip', 'os', 'channel']
>>  ['ip', 'app', 'os', 'channel']
>>  ['ip', 'app', 'os']
>>  ['ip', 'device', 'channel']
>>  ['ip', 'app', 'device', 'os', 'channel']
>>  ['device', 'channel']
>>  ['app', 'device', 'channel']
>>  ['device', 'hour']
>>  ['ip', 'device']
Aggregating by  ['ip', 'device'] ... and saved in ip_device_count
>>  ['app', 'channel']
Aggregating by  ['app', 'channel'] ... and saved in app_channel_count
>>  ['device', 'os', 'channel', 'hour']
Aggregating by  ['device', 'os', 'channel', 'hour'] ... and s

>>  ['ip', 'app'] device
Calculating variance of  device  by  ['ip', 'app'] ... and saved in ip_app_by_device_var
>>  ['app', 'os'] hour
Calculating variance of  hour  by  ['app', 'os'] ... and saved in app_os_by_hour_var
>>  ['ip', 'app'] hour
Calculating variance of  hour  by  ['ip', 'app'] ... and saved in ip_app_by_hour_var
>>  ['ip', 'app', 'device'] hour
Calculating variance of  hour  by  ['ip', 'app', 'device'] ... and saved in ip_app_device_by_hour_var
>>  ['app', 'os'] channel
Calculating variance of  channel  by  ['app', 'os'] ... and saved in app_os_by_channel_var
(18790469, 108)


,ip,app,device,os,channel,day,hour,minute,second,ip_device_prevClick,...,ip_device_os_by_hour_var,device_os_channel_by_hour_var,os_by_channel_var,app_channel_by_hour_var,ip_device_channel_by_hour_var,ip_app_by_device_var,app_os_by_hour_var,ip_app_by_hour_var,ip_app_device_by_hour_var,app_os_by_channel_var
0,5744,9,1,3,107,10,4,0,0,NaN,...,0.000000,12.514261,19569.292969,12.290454,0.333333,0.0,14.302400,13.675926,13.675926,19538.246094
1,119901,9,1,3,466,10,4,0,0,NaN,...,11.367663,14.563915,19569.292969,14.336766,13.130157,0.0,14.302400,13.528979,13.528979,19538.246094
2,72287,21,1,19,128,10,4,0,0,NaN,...,14.292033,12.521263,18658.412109,12.901376,10.612795,0.0,9.649865,7.260285,7.260285,2402.308594
3,78477,15,1,13,111,10,4,0,0,NaN,...,14.280590,14.687875,18327.267578,14.795582,0.000000,0.0,14.517140,14.722997,14.722997,17858.015625
4,123080,12,1,13,328,10,4,0,0,NaN,...,19.064627,14.444427,18327.267578,14.409562,0.000000,0.0,13.677645,10.824275,10.824275,9035.611328


Index(['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'minute',
       'second', 'ip_device_prevClick',
       ...
       'ip_device_os_by_hour_var', 'device_os_channel_by_hour_var',
       'os_by_channel_var', 'app_channel_by_hour_var',
       'ip_device_channel_by_hour_var', 'ip_app_by_device_var',
       'app_os_by_hour_var', 'ip_app_by_hour_var', 'ip_app_device_by_hour_var',
       'app_os_by_channel_var'],
      dtype='object', length=108)
data_size: 7526.39 Mb
0    0
1    1
2    2
3    3
4    4
Name: click_id, dtype: uint32
File saved: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/feature/feature_TalkingdataAFD2018_FeatureExtraction_test_20180501_143516_test.p
File loaded: 	/data1/kaggle/talkingdata-adtracking-fraud-detection/feature/feature_TalkingdataAFD2018_FeatureExtraction_test_20180501_143516_test.p
[0 1 2 3 4]
(18790469, 108)
18790469
[
  (0,	"ip")
  (1,	"app")
  (2,	"device")
  (3,	"os")
  (4,	"channel")
  (5,	"day")
  (6,	"hour")
  (7,	"minute")
  (8,	"se

In [18]:
# for key in day_rows.keys():
#     key_str = str(key)
#     print('date key: %s' % key_str)
#     if is_debug and key > 1:
#         print('is_debug=%s, skip date: %s' % (is_debug, key_str))
#         continue
#     if not is_debug and key <= 1:
#         print('is_debug=%s, skip date: %s' % (is_debug, key_str))
#         continue

In [19]:
# %%time

# for key in day_rows.keys():
#     print('*' * 80)

#     key_str = str(key)
#     print('date key: %s' % key_str)
#     if is_debug and key > 1:
#         print('is_debug=%s, skip date: %s' % (is_debug, key_str))
#         continue
#     if not is_debug and key <= 1:
#         print('is_debug=%s, skip date: %s' % (is_debug, key_str))
#         continue
    
#     n_skiprows = day_rows[key]['n_skiprows']
#     n_rows = day_rows[key]['n_rows']
    
#     train_csv = pd.read_csv(
#         train_csv_file, 
#         skiprows=range(1, n_skiprows), 
#         nrows=n_rows, 
#         usecols=train_columns,
#         dtype=dtypes,
#         parse_dates=['click_time']
#     )
    
#     print('train_csv.shape: \t\t', train_csv.shape)
#     display(train_csv.head(2))
#     print('train_csv: %.2f Mb' % (sys.getsizeof(train_csv)/1024./1024.))
# #     print('*' * 80)
    
#     y_data = train_csv['is_attributed']
#     train_csv.drop(['is_attributed'], axis=1, inplace=True)
#     display(y_data.head())

#     display(train_csv.head())
# #     print('*' * 80)

#     train_csv = do_feature(train_csv)
    
#     y_proba_file = os.path.join(feature_folder, 'feature_%s_date%s.p' % (run_name, key_str))
#     feature_files.append(y_proba_file)
#     save_feature(
#         train_csv, 
#         y_data, 
#         y_proba_file
#     )
#     x_data, y_data = load_feature(y_proba_file)

#     print(x_data.shape)
#     print(y_data.shape)
#     print('[')
#     for i, col in enumerate(train_csv.columns):
#         print('  (%s,\t"%s")' % (i, col))
#     print('')
# #     del train_csv
#     del x_data
#     del y_data
#     gc.collect()

In [20]:
# print(x_data.shape)
# print(y_data.shape)
# print(x_test.shape)
# print(click_ids.shape)
for name in feature_files:
    print(name)

/data1/kaggle/talkingdata-adtracking-fraud-detection/feature/feature_TalkingdataAFD2018_FeatureExtraction_test_20180501_143516_test.p


In [21]:
print('Time cost: %.2f s' % (time.time() - t0))
print(run_name)
print('Done!')

Time cost: 4938.03 s
TalkingdataAFD2018_FeatureExtraction_test_20180501_143516
Done!
